# Tutorial 1: The Recursion Problem

*Why Types Ban General Recursion*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-continuous-domains/blob/main/notebooks/01-the-recursion-problem.ipynb)

---

## A Note from the Archives

> *Year 848, Capital Archives*
>
> Arren Mott, three years into his appointment under Brennis Mund, was cataloguing recursive definitions when he encountered something troubling. He had been trying to type a function that grew faster than any he had seen before — a function later scholars would call Ackermann's function.
>
> The primitive recursor that Brennis had proposed could not express it. No matter how Mott arranged the types, the function escaped his grasp.
>
> "The typing discipline," Mott wrote in his notes, "has solved the Omega problem by forbidding the Y combinator. But in doing so, it has lost something essential. There are computable functions we cannot express."
>
> This was the recursion problem — and Mott would spend the next five decades solving it.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Explain why the Y combinator is essential for general recursion
2. Understand why types prevent the Y combinator
3. Recognize the expressiveness gap in typed passages
4. Appreciate why Brennis's primitive recursion is insufficient

## Setup

In [ ]:
import pandas as pd
import numpy as np

# Load datasets from central repository
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

# Load denotational semantics data
denotations_df = pd.read_csv(BASE_URL + "denotational_semantics.csv")

# Load type system comparison from previous course
comparison_df = pd.read_csv(BASE_URL + "type_system_comparison.csv")

print(f"Loaded {len(denotations_df)} denotational entries")
print(f"Loaded {len(comparison_df)} type comparisons")

## 1. The Y Combinator: The Heart of Recursion

Recall from Course 1 that the **Y combinator** enables recursive definitions:

$$Y = \lambda f. ((\lambda x. f(x\ x))(\lambda x. f(x\ x)))$$

The magic of Y is that for any function f:

$$Y\ f = f\ (Y\ f)$$

This means Y f is a **fixed point** of f. We can define factorial as:

```
factorial = Y (λf.λn. if n=0 then 1 else n * f(n-1))
```

Without Y, how would we define recursive functions?

In [ ]:
# Find Y combinator in our dataset
y_comb = denotations_df[denotations_df['expression'].str.contains('Y', na=False)]

print("Y combinator in the denotational semantics:")
for _, row in y_comb.iterrows():
    print(f"\n  Expression: {row['expression']}")
    print(f"  Type: {row['type']}")
    print(f"  Terminates: {row['terminates']}")
    print(f"  Note: {row['notes']}")

## 2. Why Types Forbid Y

Brennis Mund proved (EV-835-001) that the Y combinator cannot be typed in the simply typed lambda calculus.

The reason: Y contains the pattern $(x\ x)$ — self-application.

For $(x\ x)$ to be well-typed:
- x must have type $\tau$ (for some $\tau$)
- x must also have type $\tau \to \sigma$ (to accept itself as argument)
- So $\tau = \tau \to \sigma$
- This requires an **infinite type**: $\tau = \tau \to \tau \to \tau \to ...$

No finite type works. Y is untypable.

In [ ]:
# Self-application is the culprit
self_app = denotations_df[denotations_df['expression'].str.contains('x x', na=False)]

print("Expressions with self-application (x x):")
for _, row in self_app.head().iterrows():
    type_status = "UNTYPABLE" if row['type'] == 'untypable' else row['type']
    print(f"  {row['expression'][:40]}... -> {type_status}")

## 3. The Primitive Recursion Compromise

Brennis proposed **primitive recursion** (EV-838-001) as a typed alternative to Y:

```
rec : τ → (nat → τ → τ) → nat → τ
```

This allows recursion on natural numbers:
- Base case: `rec z f 0 = z`
- Step case: `rec z f (n+1) = f n (rec z f n)`

But there's a problem: primitive recursion is **strictly weaker** than general recursion.

In [ ]:
# Demonstrate what primitive recursion CAN express
def primitive_factorial(n):
    """Factorial via primitive recursion."""
    # rec 1 (λn.λr.r*(n+1)) n
    result = 1
    for i in range(1, n + 1):
        result = result * i
    return result

def primitive_fibonacci(n):
    """Fibonacci via primitive recursion."""
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

print("Primitive recursive functions work for simple cases:")
print(f"  factorial(5) = {primitive_factorial(5)}")
print(f"  fibonacci(10) = {primitive_fibonacci(10)}")

## 4. Ackermann's Function: Beyond Primitive Recursion

In Year 848 (EV-848-001), Mott discovered a function that primitive recursion cannot express:

$$A(m, n) = \begin{cases}
n + 1 & \text{if } m = 0 \\
A(m-1, 1) & \text{if } m > 0 \text{ and } n = 0 \\
A(m-1, A(m, n-1)) & \text{if } m > 0 \text{ and } n > 0
\end{cases}$$

This function grows faster than any primitive recursive function. It requires **general recursion**.

In [ ]:
# Ackermann's function - requires general recursion
def ackermann(m, n, depth=0, max_depth=1000):
    """Ackermann's function with recursion limit."""
    if depth > max_depth:
        return "..."  # Would take too long
    
    if m == 0:
        return n + 1
    elif n == 0:
        return ackermann(m - 1, 1, depth + 1, max_depth)
    else:
        inner = ackermann(m, n - 1, depth + 1, max_depth)
        if inner == "...":
            return "..."
        return ackermann(m - 1, inner, depth + 1, max_depth)

print("Ackermann's function grows FAST:")
print(f"  A(0, 5) = {ackermann(0, 5)}")
print(f"  A(1, 5) = {ackermann(1, 5)}")
print(f"  A(2, 5) = {ackermann(2, 5)}")
print(f"  A(3, 5) = {ackermann(3, 5)}")
print(f"  A(4, 1) = {ackermann(4, 1)}")
print(f"  A(4, 2) = {ackermann(4, 2)}  # Already huge!")

## 5. The Expressiveness Gap

Here's the fundamental tension:

| Property | Untyped λ-calculus | Simply Typed λ-calculus |
|----------|-------------------|------------------------|
| Can express Y | ✓ | ✗ |
| Can express Ackermann | ✓ | ✗ |
| Can express Omega | ✓ | ✗ |
| Always terminates | ✗ | ✓ |

Types guarantee termination but lose expressiveness.

In [ ]:
# Analyze the tradeoff
print("The Expressiveness-Safety Tradeoff:\n")

# Count typable vs untypable in our dataset
typable = denotations_df[denotations_df['type'] != 'untypable']
untypable = denotations_df[denotations_df['type'] == 'untypable']

print(f"Typable expressions: {len(typable)}")
print(f"Untypable expressions: {len(untypable)}")

# Check termination
typable_terminate = typable[typable['terminates'] == True]
untypable_terminate = untypable[untypable['terminates'] == True]

print(f"\nTypable that terminate: {len(typable_terminate)}/{len(typable)} = 100%")
print(f"Untypable that terminate: {len(untypable_terminate)}/{len(untypable)}")

## 6. Mott's Question

After discovering the recursion problem, Mott asked:

> **Can we give MEANING to recursive definitions without requiring termination?**

His insight would come seven years later (EV-855-001): 

**Non-termination is not failure — it is absence of information.**

A non-terminating computation produces no result, but that's just ⊥ (bottom) — the "undefined" value. This is still a valid mathematical object!

In the next tutorials, we'll see how this insight leads to domain theory.

In [ ]:
# Preview: Omega has a meaning
omega = denotations_df[denotations_df['expression'] == 'Ω']

if not omega.empty:
    print("Mott's insight applied to Omega:")
    print(f"\n  Expression: Ω = (λx.x x)(λx.x x)")
    print(f"  Terminates: {omega.iloc[0]['terminates']}")
    print(f"  Denotation: {omega.iloc[0]['denotation']}")
    print(f"\n  Omega MEANS something: it means ⊥ (no information)")
    print(f"  This is not an error — it's a valid semantic value!")

## Summary

In this tutorial, we learned:

1. **The Y combinator** enables general recursion through fixed points
2. **Types forbid Y** because self-application requires infinite types
3. **Primitive recursion** is a typed alternative but strictly weaker
4. **Ackermann's function** cannot be expressed with primitive recursion
5. **The expressiveness gap**: types guarantee termination but lose recursion
6. **Mott's question**: Can we give meaning to recursion without termination?

In the next tutorial, we'll see Mott's answer: the **information ordering**.

---

## Exercises

1. Try to type the expression `λx.x x`. What type would x need?

2. Write out A(2,3) step by step. How many recursive calls are made?

3. Can you think of a function that's computable but not primitive recursive (besides Ackermann)?

4. Why is "non-termination as absence of information" a useful perspective?

---

*Next: Tutorial 2 - The Information Ordering*